In [ ]:
# This command can be used to install the required packages for this project.
# !pip install -r ../requirements.txt

In [ ]:
# Add the parent directory of 'KernelCT' to sys.path

import sys
sys.path.append('..')

In [ ]:
# Import necessary packages/modules
import numpy as np
from KernelCT.object import Object
from KernelCT import utilities, phantom
from KernelCT import kernel_reconstruction as kr
from KernelCT.weighted_kernels import WeightedGaussian

In [ ]:
# Setup

## Define phatom
phantom_type = 'shepp_logan'

## Generate pixel grid
x_max = 1
y_max = 1
grid_size = [256, 256]
X, Y = utilities.generate_pixel_grid(x_max, y_max, grid_size)
phantom_eval = phantom.eval_phantom(X, Y, phantom_type)

## Create object
geometry = 'random'
r_max = np.sqrt(2)
sample_size = 10000

r, a = utilities.generate_lineset(geometry, r_max, sample_size)
Radon = phantom.eval_phantom_radon(r, a, phantom_type)
obj = Object(r, a, Radon)

## Parameters for greedy selection
kernel = WeightedGaussian()
greedy_method = 'beta_greedy'
steps = 3
steps_size = 500
beta = 0.5
c_newton = None
V_newton = None
pwr_func_vals = None
res = None

## Init kernel reconstruction
interpolant = kr.KernelInterpolant()
eval_repr = None

In [ ]:
# Init figure of reconstructions
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(24, 6))

# plot original phantom
ax = fig.add_subplot(1, steps + 1, 1)
ax.imshow(phantom_eval, cmap='gray', origin='lower')
ax.set_title('Original Phantom')
ax.axis('off')

# Run greedy kernel thinning
for i in range(steps):
    c_newton, V_newton, pwr_func_vals, res = obj.kernel_thinning(
        kernel, greedy_method, max_iter=steps_size, beta=beta,
        c_newton=c_newton, V_newton=V_newton, pwr_func_vals=pwr_func_vals,
        res=res
    )

    interpolant.fit(
        obj, kernel, basis='standard', c_newton=c_newton, V_newton=V_newton
    )

    reconstruction, eval_repr = interpolant.predict(
        obj, X, Y, kernel, prev_eval_repr=None
    )

    reconstruction_error = utilities.reconstruction_error(
        phantom_eval, reconstruction, metric='rmse'
    )

    ax = fig.add_subplot(1, steps + 1, i + 2)
    ax.imshow(reconstruction, cmap='gray', origin='lower')
    ax.set_title(f'{(i+1)*steps_size} data points - RMSE: {reconstruction_error:.4f}')
    ax.axis('off')

plt.show()